# Summary
This notebook apply the proposed method (GMM clustering) to a set of microarray datasets.

In [1]:
import sys
sys.path.append("..")
import h5py

#GPU configuration
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default


import random
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scripts.data_generator as data_generator
import scripts.feature_ranking as feature_ranking
import scripts.features_2d as features_2d
import scripts.ga as ga
import scripts.preprocess as preprocess
import scripts.ga_evaluation as ga_evaluation
import scripts.bio_analysis as bio_analysis
from sklearn import preprocessing
import tensorflow as tf
from IPython import get_ipython
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import time
import glob2

plt.ion()
plt.show()

random_state=1
random.seed( random_state )
np.random.seed(random_state)


%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



/usr/local/lib/python3.6/dist-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
import scripts.feature_sampling as feature_sampling

# Steps
- Clustering 1d to select best discriminant features

- Clustering 2d to select redundant, close and outlier features

In [3]:
import scanpy.api as sc
import scipy as sp
def filter_data(X,  highly_genes=500):
    """
    Remove less variable genes

    Args:
        X ([type]): [description]
        highly_genes (int, optional): [description]. Defaults to 500.

    Returns:
        [type]: [description]
    """

    X = np.ceil(X).astype(np.int)
    adata = sc.AnnData(X)

    sc.pp.filter_genes(adata, min_counts=3)
    sc.pp.filter_cells(adata, min_counts=1)
    sc.pp.normalize_per_cell(adata)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=4,
                                min_disp=0.5, n_top_genes=highly_genes, subset=True)
    genes_idx = np.array(adata.var_names.tolist()).astype(int)
    cells_idx = np.array(adata.obs_names.tolist()).astype(int)

    return genes_idx, cells_idx

def normalize(adata, copy=True, highly_genes = None, filter_min_counts=True, 
              size_factors=True, normalize_input=True, logtrans_input=True):
    """
    Normalizes input data and retains only most variable genes 
    (indicated by highly_genes parameter)

    Args:
        adata ([type]): [description]
        copy (bool, optional): [description]. Defaults to True.
        highly_genes ([type], optional): [description]. Defaults to None.
        filter_min_counts (bool, optional): [description]. Defaults to True.
        size_factors (bool, optional): [description]. Defaults to True.
        normalize_input (bool, optional): [description]. Defaults to True.
        logtrans_input (bool, optional): [description]. Defaults to True.

    Raises:
        NotImplementedError: [description]

    Returns:
        [type]: [description]
    """
    if isinstance(adata, sc.AnnData):
        if copy:
            adata = adata.copy()
    elif isinstance(adata, str):
        adata = sc.read(adata)
    else:
        raise NotImplementedError
    norm_error = 'Make sure that the dataset (adata.X) contains unnormalized count data.'
    assert 'n_count' not in adata.obs, norm_error
    if adata.X.size < 50e6: # check if adata.X is integer only if array is small
        if sp.sparse.issparse(adata.X):
            assert (adata.X.astype(int) != adata.X).nnz == 0, norm_error
        else:
            assert np.all(adata.X.astype(int) == adata.X), norm_error

    if filter_min_counts:
        sc.pp.filter_genes(adata, min_counts=1)#3
        sc.pp.filter_cells(adata, min_counts=1)
    if size_factors or normalize_input or logtrans_input:
        adata.raw = adata.copy()
    else:
        adata.raw = adata
    if size_factors:
        sc.pp.normalize_per_cell(adata)
        adata.obs['size_factors'] = adata.obs.n_counts / np.median(adata.obs.n_counts)
    else:
        adata.obs['size_factors'] = 1.0
    if logtrans_input:
        sc.pp.log1p(adata)
    if highly_genes != None:
        sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes = highly_genes, subset=True)
    if normalize_input:
        sc.pp.scale(adata)
    return adata

def preprocess_data(X, nb_genes = 500, normalize_input=False):
    """
    Preprocessing phase as proposed in scanpy package.
    Keeps only nb_genes most variable genes and normalizes
    the data to 0 mean and 1 std.
    Args:
        X ([type]): [description]
        nb_genes (int, optional): [description]. Defaults to 500.
    Returns:
        [type]: [description]
    """
    X = np.ceil(X).astype(np.int)
    count_X = X
    print(X.shape, count_X.shape, f"keeping {nb_genes} genes")
    orig_X = X.copy()
    adata = sc.AnnData(X)

    adata = normalize(adata,
                      copy=True,
                      highly_genes=nb_genes,
                      size_factors=True,
                      normalize_input=normalize_input,
                      logtrans_input=True)
    X = adata.X.astype(np.float32)
    return X

In [4]:
random_state=0
random.seed( random_state )
np.random.seed(random_state)


In [5]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
print(files)

['Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Diaphragm', 'mouse_ES_cell', 'Quake_Smart-seq2_Lung', 'Quake_10x_Limb_Muscle', 'mouse_bladder_cell', 'Romanov', 'Quake_Smart-seq2_Limb_Muscle', 'Muraro']


In [64]:
clustering = "gmm"
results = None
method = "adapted_ratkowsky_lance"
# imp_f = np.arange(20)
for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = 3000, normalize_input=False)
    data = preprocessing.MinMaxScaler().fit_transform(data)
    n_clusters = len(np.unique(truth))

    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=98,
                                              z_file=None,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")


    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }

    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("../reports/scRNAseq_gmm.xlsx")
    

(1350, 23341) (1350, 23341) keeping 3000 genes

##########  Quake_Smart-seq2_Trachea, (1350, 2999)
*** Computing 1D feature ranking ...


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Dispersion tests took 0.38 sec
Entropy computation 2.69 sec
KNN computation 7.21 sec
Sorting and thresholds 0.02 sec
Performing hierarchical clustering...


  0%|          | 0/31 [00:00<?, ?it/s]

Hierarchical clustering 3.68 sec
Handle redundant features 0.19 sec
Returning 376 redundant features and  45 important features
TIME: 1d Features : 0.25800102949142456 min
TIME: 2d scores: 3.973642985026042e-09 min
*** Optimization algorithm 
Non redundant features 2623, orig size 2999, nb imp : 45
adding 50 random population


...

 97%|█████████▋| 30/31 [22:04<00:38, 38.08s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.09,0.07,"[412, 500, 1773, 1791, 2251, 2635]",0.20,"[3, 3, 4, 1, 3, 3, 1, 3, 0, 3, 3, 1, 1, 1, 3, ...",0.17,0.49,6.0,"Counter({3: 805, 1: 604, 0: 347, 4: 305, 6: 30..."
1,0.09,0.14,"[213, 281, 349, 720, 1089, 1688]",0.24,"[1, 1, 1, 1, 4, 4, 2, 1, 1, 1, 1, 1, 5, 1, 3, ...",0.27,0.74,6.0,"Counter({1: 1271, 3: 446, 2: 168, 5: 108, 4: 7..."
2,0.08,0.09,"[17, 326, 965, 1051, 1397, 2064, 2449]",0.14,"[7, 0, 7, 0, 7, 0, 7, 7, 7, 7, 7, 7, 0, 4, 7, ...",0.22,0.69,7.0,"Counter({7: 1042, 0: 796, 4: 114, 6: 99, 1: 18..."
3,0.08,0.07,"[132, 558, 760, 801, 1228, 2229, 2295, 2910]",0.16,"[6, 1, 6, 6, 4, 6, 5, 1, 1, 1, 1, 1, 6, 6, 6, ...",0.20,0.56,8.0,"Counter({1: 1257, 3: 316, 6: 294, 5: 106, 7: 6..."
4,0.09,0.24,"[474, 1241, 1336, 2377, 2418, 2542]",0.36,"[1, 1, 1, 2, 1, 1, 2, 5, 1, 2, 8, 1, 0, 1, 4, ...",0.20,0.71,6.0,"Counter({1: 668, 4: 619, 0: 438, 2: 189, 5: 78..."
5,0.08,0.13,"[243, 623, 781, 2220, 2324, 2348, 2619, 2768, ...",0.23,"[0, 1, 1, 0, 7, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, ...",0.27,0.78,9.0,"Counter({0: 1042, 1: 799, 3: 136, 7: 107, 2: 1..."
6,0.08,0.24,"[256, 559, 1148, 1401, 1405, 2372]",0.31,"[5, 0, 2, 2, 1, 1, 5, 2, 2, 6, 6, 2, 2, 2, 2, ...",0.21,0.66,6.0,"Counter({1: 550, 2: 523, 7: 389, 5: 381, 0: 18..."
7,0.08,0.21,"[324, 469, 745, 1002, 1041, 1354, 1467, 1473]",0.18,"[0, 0, 0, 0, 3, 0, 0, 8, 0, 3, 3, 0, 8, 3, 0, ...",0.20,0.77,8.0,"Counter({0: 1223, 3: 666, 8: 140, 4: 44, 7: 18..."
8,0.09,0.14,"[636, 1248, 2137, 2282, 2305, 2778]",0.17,"[1, 1, 1, 1, 7, 7, 1, 1, 1, 7, 1, 1, 7, 1, 3, ...",0.25,0.84,6.0,"Counter({1: 1419, 7: 553, 3: 61, 4: 35, 8: 24,..."
9,0.10,0.45,"[373, 638, 701, 1163, 1343, 2677]",0.41,"[3, 8, 3, 3, 8, 2, 8, 2, 3, 8, 8, 3, 3, 3, 8, ...",0.21,0.56,6.0,"Counter({3: 728, 8: 648, 2: 528, 6: 103, 4: 84..."


adding 10 random population


/usr/local/lib/python3.6/dist-packages/sklearn/mixture/_base.py:148: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  random_state=random_state).fit(X).labels_
100%|██████████| 31/31 [23:39<00:00, 45.80s/it]

TIME: GA: 23.66296017964681 min
TIME: Total: 24.059157351652782 min


In [65]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
Muraro,0.45
Quake_10x_Limb_Muscle,0.28
Quake_Smart-seq2_Diaphragm,0.54
Quake_Smart-seq2_Limb_Muscle,0.14
Quake_Smart-seq2_Lung,0.25
Quake_Smart-seq2_Trachea,0.42
Romanov,0.27
mouse_ES_cell,0.32
mouse_bladder_cell,0.28


In [67]:
results = None
clustering = "gmm"

method = "adapted_ratkowsky_lance"

for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = 3000, normalize_input=True)
    data = preprocessing.MinMaxScaler().fit_transform(data)
    n_clusters = len(np.unique(truth))

    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=98,
                                              z_file=None,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")

    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }

    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)

    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("../reports/scRNAseq_gmm1.xlsx")
    

(1350, 23341) (1350, 23341) keeping 3000 genes

##########  Quake_Smart-seq2_Trachea, (1350, 2999)
*** Computing 1D feature ranking ...


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Dispersion tests took 0.43 sec
Entropy computation 2.72 sec
KNN computation 7.29 sec
Sorting and thresholds 0.02 sec
Performing hierarchical clustering...


  0%|          | 0/31 [00:00<?, ?it/s]

Hierarchical clustering 3.63 sec
Handle redundant features 0.19 sec
Returning 376 redundant features and  45 important features
TIME: 1d Features : 0.26086426178614297 min
TIME: 2d scores: 7.947285970052083e-09 min
*** Optimization algorithm 
Non redundant features 2623, orig size 2999, nb imp : 45
adding 50 random population


...

/usr/local/lib/python3.6/dist-packages/sklearn/mixture/_base.py:148: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  random_state=random_state).fit(X).labels_
 97%|█████████▋| 30/31 [20:55<00:38, 38.89s/it]

,adapted_ratkowsky_lance,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.10,0.04,"[1041, 1227, 1548, 2152]",0.12,"[0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.22,0.95,4.0,"Counter({0: 1876, 6: 176, 4: 24, 3: 23, 2: 9, ..."
1,0.09,0.32,"[589, 602, 1401, 2585, 2778, 2843]",0.30,"[5, 1, 2, 2, 4, 4, 2, 5, 2, 5, 1, 2, 5, 2, 5, ...",0.20,0.66,6.0,"Counter({2: 795, 0: 661, 1: 425, 5: 200, 7: 19..."
2,0.08,0.39,"[6, 13, 484, 924, 1223, 2605, 2682]",0.36,"[3, 2, 3, 3, 5, 3, 3, 3, 3, 0, 2, 3, 0, 3, 3, ...",0.24,0.67,7.0,"Counter({3: 864, 0: 773, 2: 262, 4: 120, 5: 41..."
3,0.08,0.06,"[277, 758, 933, 1074, 1102, 1981, 2387]",0.26,"[0, 7, 1, 3, 1, 1, 1, 8, 1, 1, 7, 1, 0, 0, 1, ...",0.26,0.76,7.0,"Counter({1: 1100, 0: 377, 7: 145, 6: 140, 2: 1..."
4,0.08,0.24,"[200, 247, 346, 629, 2226, 2251, 2686]",0.31,"[4, 1, 6, 6, 4, 4, 6, 4, 3, 1, 1, 3, 3, 6, 4, ...",0.21,0.52,7.0,"Counter({6: 588, 3: 529, 1: 345, 2: 223, 4: 21..."
5,0.09,0.07,"[719, 1064, 1144, 1317, 1705, 2323, 2623]",0.22,"[1, 0, 0, 1, 0, 0, 1, 3, 1, 0, 0, 0, 3, 1, 1, ...",0.26,0.73,7.0,"Counter({0: 975, 1: 354, 2: 326, 7: 230, 3: 21..."
6,0.08,0.06,"[474, 486, 607, 1017, 1573, 2345, 2760, 2786]",0.13,"[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, ...",0.21,0.89,8.0,"Counter({0: 1712, 2: 190, 3: 80, 1: 52, 6: 28,..."
7,0.08,0.17,"[779, 1264, 1570, 1574, 1668, 2121]",0.18,"[0, 1, 3, 0, 3, 0, 0, 0, 0, 8, 4, 0, 8, 1, 0, ...",0.26,0.80,6.0,"Counter({0: 1434, 1: 288, 3: 131, 4: 109, 8: 1..."
8,0.08,0.08,"[252, 403, 477, 749, 2274, 2282, 2633]",0.14,"[0, 5, 5, 0, 6, 1, 0, 1, 1, 5, 5, 6, 5, 0, 0, ...",0.16,0.54,7.0,"Counter({1: 797, 0: 667, 5: 525, 4: 38, 8: 36,..."
9,0.10,0.08,"[110, 1087, 1092, 1405, 2306, 2934]",0.24,"[7, 1, 0, 8, 1, 6, 1, 8, 0, 1, 8, 8, 8, 0, 8, ...",0.25,0.74,6.0,"Counter({1: 923, 0: 510, 8: 430, 7: 216, 6: 26..."


adding 14 random population


100%|██████████| 31/31 [22:23<00:00, 43.34s/it]

TIME: GA: 22.392295559247334 min
TIME: Total: 22.796740249792734 min


In [68]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
Muraro,0.39
Quake_10x_Limb_Muscle,0.32
Quake_Smart-seq2_Diaphragm,0.61
Quake_Smart-seq2_Limb_Muscle,0.22
Quake_Smart-seq2_Lung,0.27
Quake_Smart-seq2_Trachea,0.28
Romanov,0.37
mouse_ES_cell,0.19
mouse_bladder_cell,0.25


In [75]:
results.groupby("dataset_name").agg({"ari": max}).mean()

ari    0.322222
dtype: float64

In [72]:
r1 = pd.read_excel("../reports/scRNAseq_gmm.xlsx")

In [76]:
r1.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
Muraro,0.45
Quake_10x_Limb_Muscle,0.28
Quake_Smart-seq2_Diaphragm,0.54
Quake_Smart-seq2_Limb_Muscle,0.14
Quake_Smart-seq2_Lung,0.25
Quake_Smart-seq2_Trachea,0.42
Romanov,0.27
mouse_ES_cell,0.32
mouse_bladder_cell,0.28


In [ ]:
results.groupby("dataset_name").agg({"ari": max})

In [8]:
clustering = "hdbscan"
results = None
method = "adapted_silhouette"
# imp_f = np.arange(20)
for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = 3000, normalize_input=False)
    data = preprocessing.MinMaxScaler().fit_transform(data)
    n_clusters = len(np.unique(truth))

    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=98,
                                              z_file=None,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")


    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }

    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("../reports/scRNAseq_hdbscan.xlsx")
    

(1350, 23341) (1350, 23341) keeping 3000 genes

##########  Quake_Smart-seq2_Trachea, (1350, 2999)
*** Computing 1D feature ranking ...


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Dispersion tests took 0.41 sec
Entropy computation 2.7 sec
KNN computation 7.3 sec
Sorting and thresholds 0.02 sec
Performing hierarchical clustering...
Hierarchical clustering 3.67 sec


  0%|          | 0/31 [00:00<?, ?it/s]

Handle redundant features 0.23 sec
Returning 376 redundant features and  45 important features
TIME: 1d Features : 0.2684640924135844 min
TIME: 2d scores: 3.973642985026042e-09 min
*** Optimization algorithm 
Non redundant features 2623, orig size 2999, nb imp : 45
adding 50 random population


...

 97%|█████████▋| 30/31 [1:33:06<02:50, 170.37s/it]

,adapted_ratkowsky_lance,adapted_silhouette,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.01,0.895787,0.02,"[39, 67, 99, 110, 164, 192, 202, 218, 293, 297...",0.02,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1,...",0.03,0.90,102.0,"Counter({1: 1967, -1: 126, 0: 15, 2: 14})"
1,0.01,0.872513,0.05,"[6, 37, 53, 57, 106, 110, 188, 192, 260, 293, ...",0.08,"[-1, 5, 7, 7, 1, -1, 5, 7, 7, 7, -1, 7, 7, 7, ...",0.13,0.88,91.0,"Counter({7: 1822, -1: 128, 6: 46, 4: 35, 5: 26..."
2,0.01,0.889447,0.01,"[26, 57, 77, 110, 112, 118, 159, 192, 198, 218...",0.05,"[-1, 4, 4, 4, 4, 4, 4, -1, 4, 4, -1, 0, 4, 4, ...",0.11,0.90,110.0,"Counter({4: 1903, -1: 112, 3: 42, 2: 21, 5: 16..."
3,0.01,0.862375,0.07,"[35, 104, 218, 297, 370, 394, 409, 412, 421, 4...",0.13,"[7, 2, 7, 7, 0, -1, 7, 7, 7, 7, 7, 7, 7, 7, 7,...",0.09,0.88,60.0,"Counter({7: 1829, -1: 134, 2: 51, 6: 30, 1: 19..."
4,0.01,0.860750,0.05,"[21, 39, 61, 110, 159, 168, 192, 202, 292, 293...",0.06,"[3, -1, 3, 3, 3, 3, 3, 2, 3, 3, 3, -1, 3, 3, 3...",0.07,0.87,100.0,"Counter({3: 1843, -1: 163, 4: 53, 2: 26, 1: 26..."
5,0.01,0.880600,0.01,"[49, 68, 90, 99, 110, 164, 192, 206, 293, 295,...",0.04,"[4, 2, 4, 1, 4, 4, 4, 4, 4, 4, 3, 4, 5, 3, 1, ...",0.10,0.89,104.0,"Counter({4: 1875, -1: 128, 5: 32, 3: 31, 1: 28..."
6,0.01,0.878809,0.03,"[3, 6, 68, 99, 186, 192, 202, 204, 227, 260, 3...",0.06,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...",0.15,0.89,70.0,"Counter({8: 1839, -1: 119, 9: 33, 3: 29, 7: 24..."
7,0.01,0.857431,0.04,"[37, 39, 99, 105, 110, 123, 129, 159, 164, 186...",0.08,"[5, 5, 5, 5, -1, 5, 5, 5, 5, 5, 5, -1, 5, 5, 5...",0.11,0.86,121.0,"Counter({5: 1820, -1: 165, 2: 33, 1: 28, 4: 27..."
8,0.01,0.860407,0.02,"[21, 57, 68, 100, 112, 121, 123, 130, 134, 168...",0.05,"[-1, 5, 5, 5, 5, -1, 5, 5, 5, 5, 5, 5, 5, 5, 5...",0.09,0.87,94.0,"Counter({5: 1825, -1: 158, 3: 38, 4: 31, 6: 22..."
9,0.01,0.875987,0.04,"[26, 154, 168, 188, 192, 198, 218, 243, 250, 2...",0.08,"[2, 2, 2, 2, -1, -1, 2, 2, 2, 2, 2, 2, 2, 2, 2...",0.06,0.89,89.0,"Counter({2: 1889, -1: 138, 1: 38, 0: 32, 3: 25})"


adding 41 random population


100%|██████████| 31/31 [1:40:36<00:00, 194.72s/it]


TIME: GA: 100.60301099618276 min
TIME: Total: 101.00193509658178 min


In [9]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
Muraro,0.07
Quake_10x_Limb_Muscle,0.02
Quake_Smart-seq2_Diaphragm,0.08
Quake_Smart-seq2_Limb_Muscle,0.06
Quake_Smart-seq2_Lung,0.07
Quake_Smart-seq2_Trachea,0.12
Romanov,0.04
mouse_ES_cell,-0.01
mouse_bladder_cell,0.01


In [6]:
clustering = "leiden"
results = None
method = "adapted_silhouette"
# imp_f = np.arange(20)
for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = 3000, normalize_input=False)
    data = preprocessing.MinMaxScaler().fit_transform(data)
    n_clusters = len(np.unique(truth))

    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=98,
                                              z_file=None,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")


    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 3
    epochs = 10*round_size

    sampling = {
    "ARCHIVE2D": { 
        "ga": 0,
        "max": 0 },
    "CLOSE": { 
        "ga": 0.35,
        "max": 0.35 },
    "IMP1D": { 
        "ga": 0.35,
        "max": 0.35 },
    "RANDOM": { 
        "ga": 0.3,
        "max": 0.3},
    }

    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=None,#population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("../reports/scRNAseq_leiden.xlsx")
    

(1350, 23341) (1350, 23341) keeping 3000 genes

##########  Quake_Smart-seq2_Trachea, (1350, 2999)
*** Computing 1D feature ranking ...


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Dispersion tests took 0.53 sec
Entropy computation 2.77 sec
KNN computation 7.68 sec
Sorting and thresholds 0.02 sec
Performing hierarchical clustering...
Hierarchical clustering 4.08 sec


  0%|          | 0/31 [00:00<?, ?it/s]

Handle redundant features 0.21 sec
Returning 376 redundant features and  45 important features
TIME: 1d Features : 0.2835372567176819 min
TIME: 2d scores: 3.973642985026042e-09 min
*** Optimization algorithm 
Non redundant features 2623, orig size 2999, nb imp : 45
adding 50 random population


 10%|▉         | 3/31 [19:18<2:54:19, 373.56s/it]

,adapted_ratkowsky_lance,adapted_silhouette,ari,features,nmi,partition,point_biserial,silhouette,size,structure
0,0.01,0.860828,0.1,"[40, 101, 102, 116, 136, 185, 193, 260, 262, 2...",0.07,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.27,0.87,76.0,"Counter({0: 1209, 1: 53, 2: 37, 3: 28, 4: 10, ..."


adding 20 random population


 16%|█▌        | 5/31 [1:03:46<5:31:36, 765.24s/it]


KeyboardInterrupt: 

In [10]:
# ! mv ../reports/scRNAseq_leiden.xlsx ../reports/scRNAseq_hdbscan.xlsx

# Supervised analysis of datasets

In [ ]:
import scripts.ga_evaluation as ga_evaluation

imp_f = np.arange(20)
result_df = pd.DataFrame()

for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = data.shape[1], normalize_input=False)


    n_clusters = len(np.unique(truth))

    result = {"Dataset" : name,
             "Original Dimensions": f"{data.shape[0]} x {data.shape[1]}",
             "Cluster sizes" : ", ".join(np.array(list(Counter(truth).values())).astype(str))}
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data).labels_
    ari_all = adjusted_rand_score(truth, predK)
    
    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(data[:, :5]).labels_
    ari_top10 = adjusted_rand_score(truth, predK)
    
    pca = PCA(2)
    pca_data = pca.fit_transform(data)

    predK = KMeans(n_clusters= n_clusters, random_state = 2).fit(pca_data).labels_
    ari_pca = adjusted_rand_score(truth, predK)
    
    r1 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "gmm")
    r2 = ga_evaluation.random_sampling(data, truth, n_clusters, algo = "hdbscan")
    result["ARI all dataset"] = round(ari_all,2)
    result["ARI PCA dataset"] = round(ari_pca,2)
    result["ARI top 10 features"] = round(ari_top10,2)
    result["Random GMM"] = round(r1,2)
    result["Random HDBSCAN"] = round(r2,2)
    result_df = result_df.append(result, ignore_index = True)

result_df

# Run time analysis

In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

In [ ]:
results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

In [ ]:
perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

In [ ]:
perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})

In [ ]:
plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')

# Best scores using supervised feature selection

In [10]:
import scripts.ga_evaluation as ga_evaluation
from sklearn import mixture
import hdbscan


In [15]:
for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    if truth.min()>0:
        truth = truth - truth.min()
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = data.shape[1], normalize_input=False)
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    features = feature_ranking.supervised_feature_ranking(data, truth,
                                               nbTopFeatures=10, random_state=42)

    input_data = data[:, features]
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    gmm_scores.append(ari)

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")
    

(1350, 23341) (1350, 23341) keeping 23341 genes
Counter({3: 830, 0: 206, 2: 201, 1: 113})
[16:38:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.



Quake_Smart-seq2_Trachea GMM ari = 0.5267989404777248, 
Quake_Smart-seq2_Trachea HDBSCAN ari = 0.08320905817621092, 
(870, 23341) (870, 23341) keeping 23341 genes
Counter({4: 439, 3: 241, 1: 81, 0: 78, 2: 31})
[16:39:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.



Quake_Smart-seq2_Diaphragm GMM ari = 0.7813372139535002, 
Quake_Smart-seq2_Diaphragm HDBSCAN ari = 0.7738596239403225, 
(2717, 24175) (2717, 24175) keeping 24175 genes
Counter({0.0: 933, 3.0

# Mutual Information

In [8]:
from sklearn.feature_selection import chi2,  mutual_info_classif, SelectKBest

In [9]:
for name in files:
    t1 = time.time()
    data_mat = h5py.File(f"{path}/real_data/{name}.h5", "r")

    truth = np.array(data_mat['Y'])
    data = np.array(data_mat['X'])
    data = preprocess_data(data, nb_genes = data.shape[1], normalize_input=False)
    print(Counter(truth))
    n_clusters = len(np.unique(truth))
    gmm_scores = []
    hdbscan_scores = []
    sel = SelectKBest(mutual_info_classif, k=50).fit_transform(data, truth)
    for i in range(2, 50):
        input_data = sel[:, :i]
        gmm = mixture.GaussianMixture(n_components=n_clusters,
                          covariance_type="full", random_state=0)
        pred = gmm.fit_predict(input_data)
        ari = adjusted_rand_score(truth, pred)
        gmm_scores.append(ari)

        pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
        ari = adjusted_rand_score(truth, pred)
        hdbscan_scores.append(ari)

        
    print(f"\n\n\n{name} GMM ari = {max(gmm_scores)}, ")
    print(f"{name} HDBSCAN ari = {max(hdbscan_scores)}, ")

(1350, 23341) (1350, 23341) keeping 23341 genes
Counter({3: 830, 0: 206, 2: 201, 1: 113})



Quake_Smart-seq2_Trachea GMM ari = 0.6129149806343049, 
Quake_Smart-seq2_Trachea HDBSCAN ari = 0.9008038537712834, 
(870, 23341) (870, 23341) keeping 23341 genes
Counter({4: 439, 3: 241, 1: 81, 0: 78, 2: 31})



Quake_Smart-seq2_Diaphragm GMM ari = 0.9732022491521348, 
Quake_Smart-seq2_Diaphragm HDBSCAN ari = 0.9637359652718737, 
(2717, 24175) (2717, 24175) keeping 24175 genes
Counter({1.0: 933, 4.0: 798, 3.0: 683, 2.0: 303})



mouse_ES_cell GMM ari = 0.8273002091209083, 
mouse_ES_cell HDBSCAN ari = 0.6575234314031914, 
(1676, 23341) (1676, 23341) keeping 23341 genes
Counter({6: 693, 10: 423, 4: 113, 3: 90, 8: 85, 7: 65, 0: 57, 1: 53, 9: 37, 5: 35, 2: 25})



Quake_Smart-seq2_Lung GMM ari = 0.7520819033960576, 
Quake_Smart-seq2_Lung HDBSCAN ari = 0.8718218369984635, 
(3909, 23341) (3909, 23341) keeping 23341 genes
Counter({2: 1330, 4: 1136, 0: 461, 5: 354, 1: 320, 3: 308})



Quake_10x_Limb_Mu

In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(["khan","west",  "gordon",'borovecki'])
path = '../data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values

    n_clusters = len(np.unique(truth))
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")

    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")

In [ ]:
import scripts.ga_evaluation as ga_evaluation
filenames = np.array(['borovecki'])
path = '../data/microarray/'

for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    input_data = data.drop("truth", axis = 1).values
    input_data = input_data[:, :10000]
    

    n_clusters = len(np.unique(truth))
    
    pred = hdbscan.HDBSCAN(min_cluster_size =2).fit(input_data).labels_
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} HDBSCAN ari = {ari}")
    
    gmm = mixture.GaussianMixture(n_components=n_clusters,
                      covariance_type="full", random_state=0)
    pred = gmm.fit_predict(input_data)
    ari = adjusted_rand_score(truth, pred)
    print(f"{name} GMM ari = {ari}")




# Execution time analysis


In [ ]:
results = pd.read_excel("reports/r_cnn.xlsx", index_col=0)

results["label"] = results["dataset_name"] + " (" +results["input_size"].astype(str) + " features) " 

perf = results.groupby("input_size").min()[['t(feature_sel)', 't(cnn)','t(ga)']]

perf = perf.rename(columns = {
    't(feature_sel)' : '1D Feature ranking', 
    't(cnn)' : '2D Feature ranking with NN',
    't(ga)': 'Optimization algorithm for top 10 subspaces'
})



plt.figure(figsize = (10,3))
ax = plt.gca()
perf.plot(kind='bar', stacked=True, ax = ax)
plt.ylabel("time (min)")
plt.xticks(rotation = 0)
sns.despine()
plt.title("Run times on West, Khan, Gordon and Boroveki datasets")
plt.xlabel("Number of dimensions in the input dataset")
plt.savefig(f"images/run_times.pdf", bbox_inches='tight')